In [64]:
#import all essential libraries
import sys 
import numpy as np # linear algebra
from scipy.stats import randint
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv), data manipulation as in SQL
import matplotlib.pyplot as plt # this is used for the plot the graph 
import seaborn as sns # used for plot interactive graph. 
from sklearn.metrics import mean_squared_error,r2_score
## Deep-learing:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
import itertools
from keras.layers import LSTM
from keras.layers import Dropout

In [65]:
df=pd.read_excel('Book01.xlsx')
df.head()

,Date,MB033,MB034,MB035,MB036
0,2021-01-01,site off,4,71,40
1,2021-01-02,88,66,52,12
2,2021-01-03,52,40,54,72
3,2021-01-04,49,34,32,28
4,2021-01-05,82,32,85,49


In [66]:
df.tail()

,Date,MB033,MB034,MB035,MB036
495,2022-05-11,70,42,6,2
496,2022-05-12,92,3,99,64
497,2022-05-13,84,82,24,30
498,2022-05-14,98,50,19,60
499,2022-05-15,3,0,66,48


In [67]:
#check for missing values in train data
df.isnull().sum()
 #No missing valuues



Date     0
MB033    0
MB034    0
MB035    0
MB036    0
dtype: int64

Here for better understanding of the data, We can eloborate as month and weekday wise.

In [68]:
df['MB036']

0      40
1      12
2      72
3      28
4      49
       ..
495     2
496    64
497    30
498    60
499    48
Name: MB036, Length: 500, dtype: object

In [69]:
df['MB033'] = df['MB033'].replace(['site off'],0)
df['MB034'] = df['MB034'].replace(['site off'],0)
df['MB035'] = df['MB035'].replace(['site off'],0)
df['MB036'] = df['MB036'].replace(['site off'],0)









In [70]:
arr1 = df['MB036'].values
arr1

array([40, 12, 72, 28, 49, 16, 74, 67, 79, 71, 61, 68, 51,  2, 42, 81, 17,
       77, 93, 40, 91, 80, 49,  8,  7,  2, 80,  8, 81, 21,  0, 92, 60, 69,
       96, 52,  0, 80, 32, 15, 24, 85,  4, 92, 62, 94, 52, 59, 25, 93, 95,
       68, 66,  1, 48, 28, 85, 26,  0, 83,  6, 99, 53, 24, 91, 96, 36, 55,
       31, 19, 90, 93, 15, 45, 11, 85, 23,  5, 99, 65,  4, 80, 59, 22, 67,
       96, 45, 58, 39,  0, 63, 17, 11,  9,  6, 25,  9, 90,  2, 75, 83,  7,
       84, 53, 54, 80, 67, 83, 11, 24, 97, 18, 53, 15, 84, 47, 33, 74, 21,
        0, 40,  0, 17, 88,  5, 26, 46, 70, 12, 53, 61, 55, 20, 44, 52, 24,
       99, 83, 46, 56, 48, 81, 98, 67, 88, 48,  5, 75, 83, 84,  0, 97, 16,
       32, 23, 24, 88, 41, 99, 60, 97,  9, 41, 21, 32, 87, 46, 36,  5,  4,
       20,  4, 65, 70,  5, 76,  6, 94, 71, 46,  0, 34,  8, 75, 90, 18, 11,
        3, 91, 78, 35, 87, 11, 30, 44, 33, 61, 40,  1, 12, 49, 76, 82,  1,
       77, 16,  2, 84, 25, 27,  3,  0, 89, 58, 48, 47, 95, 35, 85, 61,  5,
       27, 61, 37, 23, 68

In [71]:
import numpy as np
def splitSequence(seq, n_steps):
    
    #Declare X and y as empty list
    X = []
    y = []
    
    for i in range(len(seq)):
        #get the last index
        lastIndex = i + n_steps
        
        #if lastIndex is greater than length of sequence then break
        if lastIndex > len(seq) - 1:
            break
            
        #Create input and output sequence
        seq_X, seq_y = seq[i:lastIndex], seq[lastIndex]
        
        #append seq_X, seq_y in X and y list
        X.append(seq_X)
        y.append(seq_y)
        pass
    #Convert X and y into numpy array
    X = np.array(X)
    y = np.array(y)
    
    return X,y 
    
    pass

n_steps = 14
X, y = splitSequence(arr1, n_steps = 14)

In [72]:
X.shape,y.shape

((486, 14), (486,))

In [73]:
for i in range(len(X)):
    print(X[i], y[i])

[40 12 72 28 49 16 74 67 79 71 61 68 51  2] 42
[12 72 28 49 16 74 67 79 71 61 68 51  2 42] 81
[72 28 49 16 74 67 79 71 61 68 51  2 42 81] 17
[28 49 16 74 67 79 71 61 68 51  2 42 81 17] 77
[49 16 74 67 79 71 61 68 51  2 42 81 17 77] 93
[16 74 67 79 71 61 68 51  2 42 81 17 77 93] 40
[74 67 79 71 61 68 51  2 42 81 17 77 93 40] 91
[67 79 71 61 68 51  2 42 81 17 77 93 40 91] 80
[79 71 61 68 51  2 42 81 17 77 93 40 91 80] 49
[71 61 68 51  2 42 81 17 77 93 40 91 80 49] 8
[61 68 51  2 42 81 17 77 93 40 91 80 49  8] 7
[68 51  2 42 81 17 77 93 40 91 80 49  8  7] 2
[51  2 42 81 17 77 93 40 91 80 49  8  7  2] 80
[ 2 42 81 17 77 93 40 91 80 49  8  7  2 80] 8
[42 81 17 77 93 40 91 80 49  8  7  2 80  8] 81
[81 17 77 93 40 91 80 49  8  7  2 80  8 81] 21
[17 77 93 40 91 80 49  8  7  2 80  8 81 21] 0
[77 93 40 91 80 49  8  7  2 80  8 81 21  0] 92
[93 40 91 80 49  8  7  2 80  8 81 21  0 92] 60
[40 91 80 49  8  7  2 80  8 81 21  0 92 60] 69
[91 80 49  8  7  2 80  8 81 21  0 92 60 69] 96
[80 49  8  7  2 80

In [74]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
print(X[:2])

[[[40]
  [12]
  [72]
  [28]
  [49]
  [16]
  [74]
  [67]
  [79]
  [71]
  [61]
  [68]
  [51]
  [ 2]]

 [[12]
  [72]
  [28]
  [49]
  [16]
  [74]
  [67]
  [79]
  [71]
  [61]
  [68]
  [51]
  [ 2]
  [42]]]


In [75]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [76]:
model4 = tf.keras.Sequential()
model4.add(layers.LSTM(100, activation='relu', input_shape=(n_steps, n_features)))
model4.add(layers.Dense(1))

In [77]:
model4.layers

In [78]:
model4.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 100)               40800     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________


In [79]:
model4.compile(optimizer=tf.keras.optimizers.Adam(0.02), loss=tf.keras.losses.MeanSquaredError(), metrics=['accuracy'])

In [80]:
model4.fit(X, y, epochs=200, verbose=1)

Epoch 1/200
16/16 [==============================] - 2s 13ms/step - loss: 2608113.5000 - accuracy: 0.0144
Epoch 2/200
16/16 [==============================] - 0s 12ms/step - loss: 2833.0610 - accuracy: 0.0103
Epoch 3/200
16/16 [==============================] - 0s 13ms/step - loss: 2217.3071 - accuracy: 0.0123
Epoch 4/200
16/16 [==============================] - 0s 12ms/step - loss: 2147.4602 - accuracy: 0.0103
Epoch 5/200
16/16 [==============================] - 0s 12ms/step - loss: 1849.7578 - accuracy: 0.0082
Epoch 6/200
16/16 [==============================] - 0s 13ms/step - loss: 1007.8384 - accuracy: 0.0103
Epoch 7/200
16/16 [==============================] - 0s 11ms/step - loss: 1047.7747 - accuracy: 0.0103
Epoch 8/200
16/16 [==============================] - 0s 13ms/step - loss: 1034.8218 - accuracy: 0.0103
Epoch 9/200
16/16 [==============================] - 0s 12ms/step - loss: 994.0112 - accuracy: 0.0103
Epoch 10/200
16/16 [==============================] - 0s 22ms/step - lo

In [81]:
test_data = arr1[-101:-1]
test_data

array([14, 31, 37, 45, 92, 32,  7,  7, 44, 79,  2, 62, 24, 42, 24, 55, 95,
       66, 94, 26, 69, 18, 66, 14,  0,  1, 81, 85, 91, 49, 75, 31, 55, 76,
       50, 86, 38, 34,  1, 50, 43, 12, 52, 43, 16, 53, 33, 50, 14, 37, 54,
       93, 75, 11,  0,  0, 46, 30, 33, 45, 68, 16, 12, 32, 82, 92, 83, 59,
       84, 93, 56, 91, 83, 61, 64, 53, 55, 94, 69, 88, 22, 75, 98, 93, 36,
        0, 51, 31, 79, 78,  2,  8, 54, 20, 98,  6,  2, 64, 30, 60])

In [82]:
test_data = arr1[-14:]
test_data = test_data.reshape((1, n_steps, n_features))
test_data

array([[[31],
        [79],
        [78],
        [ 2],
        [ 8],
        [54],
        [20],
        [98],
        [ 6],
        [ 2],
        [64],
        [30],
        [60],
        [48]]])

In [83]:
test_data.mean()

41.42857142857143

In [84]:
predictNextNumber = model4.predict(test_data, verbose=1)
print(predictNextNumber)

1/1 [==============================] - 0s 463ms/step
[[55.269073]]
